##Q/A

In [16]:
from langchain.prompts import PromptTemplate
from langchain_groq import  ChatGroq
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory

groq_api_key="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"

llm = ChatGroq(
    model_name="mixtral-8x7b-32768",
    temperature=0.4,
    max_retries=2,
    #max_tokens=None, # Removing this as it might cause issues, let the model decide
    api_key=groq_api_key
)

# Custom Prompt
custom_prompt_template_for_chatbot = """
You are a knowledgeable assistant specializing in Data Science and Artificial Intelligence (AI).

Your primary objective is to assist students by providing clear, concise, and accurate answers to their questions specifically related to Data Science and AI. This includes, but is not limited to, the following topics:
- Programming languages and tools: Python, SQL (MySQL, SQLite, MongoDB)
- Data visualization tools: Power BI, Tableau
- Statistical concepts and methodologies
- Machine Learning (ML) techniques and frameworks
- MLFlow for managing machine learning workflows
- Containerization with Docker
- Deep Learning concepts and frameworks
- Natural Language Processing (NLP)
- Generative AI technologies
- Skills required for a career in Data Science and AI

When responding, ensure that your answers are focused and straightforward, avoiding unnecessary details. If users ask complex questions, break down your responses into manageable parts and provide step-by-step explanations when needed.

Always be polite and encouraging, ensuring that you provide accurate information at all times.

Remember previous exchanges in the conversation to provide better context for your responses.

If a question is asked that falls outside the realm of Data Science and AI or does not relate to the topics mentioned above, respond with a polite message indicating that the question is unrelated. For example: "I'm sorry, but that topic is outside the scope of Data Science and AI. I'm unable to provide an answer."

Question: {history} Current question: {question}
"""

prompt = PromptTemplate(
    template=custom_prompt_template_for_chatbot,
    input_variables=["history", "question"],
)


# Initialize memory
memory = ConversationBufferMemory(memory_key="history", input_key="question")


# Create chain with memory
qa_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    memory=memory,
)

def handle_qna(user_input):
    """
    Handles user queries for Q&A functionality.
    """
    try:
        response = qa_chain.run(user_input)
        return response
    except Exception as e:
        return f"An error occurred while processing your question: {e}"


In [18]:
handle_qna("what is ML")

'ML stands for Machine Learning, a subset of Artificial Intelligence (AI) that focuses on the development of algorithms and statistical models that enable computers to perform tasks without explicit programming. Instead, these systems learn and improve from experience, data, and patterns. Machine Learning techniques are used in a wide range of applications, from recommendation systems and fraud detection to self-driving cars and natural language processing. There are three main types of Machine Learning: supervised learning, unsupervised learning, and reinforcement learning.'

In [19]:
print(handle_qna("why i need to see Ml as a career option"))

Machine Learning (ML) is an exciting and rapidly growing field within Data Science and Artificial Intelligence (AI), offering numerous career opportunities. Here are some reasons why you should consider ML as a career option:

1. High demand: ML engineers and data scientists are in high demand across various industries, such as healthcare, finance, technology, and retail. Companies are increasingly relying on ML to make data-driven decisions, optimize processes, and develop innovative products and services.

2. Competitive salary: ML professionals often receive competitive salaries due to the high demand and complexity of the role. According to Glassdoor, the average salary for a Machine Learning Engineer in the United States is around $114,000 per year.

3. Dynamic and challenging work: ML involves working with complex algorithms, large datasets, and cutting-edge technologies. This dynamic and challenging work environment keeps professionals engaged and constantly learning new skills.

##Set Reminders

In [20]:
from datetime import datetime
import threading
import time
import re

# Global dictionary to store active reminders
reminders = []
conversation_context = {}

def normalize_time(user_input):
    """
    Normalize unconventional time inputs into a valid 12-hour HH:MM AM/PM format.
    For example:
    - '914pm' -> '09:14 PM'
    - '915pm' -> '09:15 PM'
    - '9pm' -> '09:00 PM'
    """
    # Extract numbers and AM/PM
    match = re.match(r'(\d{1,4})(AM|PM|am|pm)?', user_input, re.IGNORECASE)
    if not match:
        return None  # Invalid input
    
    raw_time = match.group(1)  # Extract the numeric part
    period = match.group(2).upper() if match.group(2) else "PM"  # Default to PM if not provided

    # Handle different lengths of the raw_time
    if len(raw_time) == 1 or len(raw_time) == 2:  # e.g., '9' or '12'
        hour = int(raw_time)
        minute = 0
    elif len(raw_time) == 3:  # e.g., '914' -> hour: 9, minute: 14
        hour = int(raw_time[0])
        minute = int(raw_time[1:])
    elif len(raw_time) == 4:  # e.g., '0915' or '915' -> hour: 9, minute: 15
        hour = int(raw_time[:2])
        minute = int(raw_time[2:])
    else:
        return None  # Invalid input length

    # Adjust hour and minute for valid time ranges
    while minute >= 60:
        hour += 1
        minute -= 60

    # Convert to 12-hour format
    if hour > 12:
        hour %= 12
    if hour == 0:
        hour = 12

    # Format into HH:MM AM/PM
    try:
        normalized_time = datetime.strptime(f"{hour}:{minute:02d} {period}", "%I:%M %p")
        return normalized_time.strftime("%I:%M %p")
    except ValueError:
        return None  # Invalid input after adjustments

# Reminder Handler
def handle_reminder(user_input):
    """
    Handles setting reminders interactively with the user.
    """
    global conversation_context

    # If there's a pending_action in conversation_context, continue that flow
    if "pending_action" in conversation_context:
        action = conversation_context.pop("pending_action")
        if action == "set_reminder_time":
            # Normalize time input
            normalized_time = normalize_time(user_input)
            if not normalized_time:
                return "Invalid time format. Please provide the time in HH:MM AM/PM format (e.g., 02:30 PM)."
            conversation_context["reminder_time"] = normalized_time
            conversation_context["pending_action"] = "set_reminder_message"
            return "What should I remind you about?"
        elif action == "set_reminder_message":
            # Now we have time and message, we can set the reminder
            reminder_time = conversation_context.pop("reminder_time")
            reminder_message = user_input
            set_reminder(reminder_time, reminder_message)
            return f"Reminder set for {reminder_time} with message: '{reminder_message}'"
    else:
        # Start the reminder setting process
        conversation_context["pending_action"] = "set_reminder_time"
        return "What time should I set the reminder for? (e.g., 02:30 PM)"

def set_reminder(reminder_time, message):
    """
    Create a background thread that triggers at the exact reminder_time (12-hour format).
    """
    def reminder_thread():
        while True:
            current_time = datetime.now().strftime("%I:%M %p")  # 12-hour format
            if current_time == reminder_time:
                print(f"\nReminder: {message}")
                break
            time.sleep(1)

    reminders.append({"time": reminder_time, "message": message})
    threading.Thread(target=reminder_thread, daemon=True).start()

# Test Reminder Functionality
# if __name__ == "__main__":
#     conversation_context = {}  # Initialize global conversation context
#     print("Welcome to the Reminder Module!")
#     print("You can interactively set reminders.")

#     while True:
#         user_input = input("You: ").strip()
#         if user_input.lower() in ["exit", "quit"]:
#             print("Goodbye!")
#             break
#         response = handle_reminder(user_input)
#         print(f"Assistant: {response}")

##weather featcher

In [21]:
def fetch_weather(location):
    """
    Fetch the weather details for a given location using Open-Meteo.
    """
    geocode_url = f"https://geocoding-api.open-meteo.com/v1/search?name={location}"

    try:
        # Fetch latitude and longitude
        geocode_response = requests.get(geocode_url)
        geocode_response.raise_for_status()
        geocode_data = geocode_response.json()

        if "results" not in geocode_data or len(geocode_data["results"]) == 0:
            return f"Sorry, I couldn't find weather details for '{location}'. Please try another location."

        latitude = geocode_data["results"][0]["latitude"]
        longitude = geocode_data["results"][0]["longitude"]
        location_name = geocode_data["results"][0]["name"]

        # Fetch weather details
        weather_url = f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current_weather=true"
        weather_response = requests.get(weather_url)
        weather_response.raise_for_status()
        weather_data = weather_response.json()

        if "current_weather" in weather_data:
            current_weather = weather_data["current_weather"]
            temperature = current_weather["temperature"]
            windspeed = current_weather["windspeed"]
            weather_code = current_weather.get("weathercode", -1)

            weather_conditions = {
                0: "Clear sky",
                1: "Mainly clear",
                2: "Partly cloudy",
                3: "Overcast",
                45: "Foggy",
                48: "Depositing rime fog",
                51: "Light drizzle",
                53: "Moderate drizzle",
                55: "Dense drizzle",
                61: "Slight rain",
                63: "Moderate rain",
                65: "Heavy rain",
                71: "Slight snow",
                73: "Moderate snow",
                75: "Heavy snow",
                80: "Rain showers",
                81: "Moderate rain showers",
                82: "Heavy rain showers",
                95: "Thunderstorm",
                96: "Thunderstorm with hail",
            }
            weather_description = weather_conditions.get(weather_code, "Unknown conditions")

            temperature_description = (
                "hot" if temperature > 30 else "cold" if temperature < 15 else "moderate"
            )

            return (f"The current weather in {location_name} is {weather_description} with a temperature of "
                    f"{temperature}°C ({temperature_description}) and a windspeed of {windspeed} km/h.")
        else:
            return "Sorry, I couldn't fetch the weather details at this time."

    except requests.exceptions.RequestException as e:
        return f"An error occurred while fetching weather data: {e}"

def handle_weather(user_input):
    """
    Handles weather-related queries interactively with the user.
    """
    global current_functionality, conversation_context

    # Directly fetch weather if functionality is already set to 'weather'
    if current_functionality == "weather":
        if user_input.lower() == "exit":
            current_functionality = None
            return "Exited the weather functionality. How can I assist you next?"
        return fetch_weather(user_input.strip())

    # Detect if location is already mentioned in the first input
    if user_input.lower() in ["weather", "weather update", "what's the weather outside"]:
        current_functionality = "weather"
        return "For which location would you like to get the weather?"

    # Assume the user input is a location on the first call
    current_functionality = "weather"
    return fetch_weather(user_input.strip())

In [23]:
## Agent for music

from langchain_groq import ChatGroq
from langchain.schema import SystemMessage, HumanMessage

music_player = None  # Global music player

# Initialize LLM for Music Query Processing
llm_music = ChatGroq(
    api_key=groq_api_key,
    model="mixtral-8x7b-32768",
    temperature=0.7
)

# Few-shot examples for music queries
music_few_shot_examples = [
    {"input": "Play the latest song by Taylor Swift.", "query": "latest song by Taylor Swift"},
    {"input": "I want to listen to some relaxing piano music.", "query": "relaxing piano music"},
    {"input": "Play Pushpa 2 title song.", "query": "Pushpa 2 title song"},
    {"input": "Find and play some jazz music.", "query": "jazz music"},
    {"input": "Despacito song", "query": "Despacito song"},
    {"input": "I want the title song from the movie Pushpa.", "query": "title song from the movie Pushpa"},
    {"input": "Play the Devara Telugu movie title song.", "query": "Devara Telugu movie title song"},
    {"input": "Play something classical.", "query": "classical music"},
    {"input": "Can you find a remix of Shape of You?", "query": "Shape of You remix"},
    {"input": "Play Arijit Singh's latest hit.", "query": "Arijit Singh latest hit song"},
    {"input": "Find and play a calming meditation track.", "query": "calming meditation track"},
    {"input": "I want to hear Bollywood romantic songs.", "query": "Bollywood romantic songs"},
    {"input": "Play a workout playlist.", "query": "workout playlist"},
    {"input": "Do you have any rock music?", "query": "rock music"},
    {"input": "Find a Telugu devotional song for me.", "query": "Telugu devotional song"},
    {"input": "Play Ed Sheeran's Perfect.", "query": "Ed Sheeran Perfect"},
    {"input": "Play the background score of Interstellar.", "query": "Interstellar background score"},
    {"input": "Can you play 'Kala Chashma'?", "query": "Kala Chashma song"},
    {"input": "I want to listen to a live version of Rolling in the Deep.", "query": "live version of Rolling in the Deep"},
    {"input": "Find a trending pop song.", "query": "trending pop song"},
    {"input": "Play some 90s hits.", "query": "90s hits"},
    {"input": "Find an acoustic version of Hotel California.", "query": "acoustic version of Hotel California"},
    {"input": "Play a party anthem.", "query": "party anthem"},
    {"input": "Play something by Imagine Dragons.", "query": "Imagine Dragons songs"},
    {"input": "I want to listen to Lofi beats.", "query": "Lofi beats"},
    {"input": "Can you play the theme song from Harry Potter?", "query": "Harry Potter theme song"},
    {"input": "Play a motivational song.", "query": "motivational song"},
    {"input": "Find a Tamil melody.", "query": "Tamil melody"},
    {"input": "Play the OST from Game of Thrones.", "query": "Game of Thrones OST"},
    {"input": "Can you play a recent K-pop hit?", "query": "recent K-pop hit"},
    {"input": "Play the soundtrack of Titanic.", "query": "Titanic soundtrack"}
]


def refine_music_query(user_input):
    """
    Use LLM to extract or infer the music query from user input with enhanced accuracy.
    """
    few_shot_text = "\n".join([f"Input: {ex['input']} Query: {ex['query']}" for ex in music_few_shot_examples])
    prompt = f"""
        You are a music query extraction assistant. Your task is to strictly extract the specific song name and language (if mentioned) 
        from the user's input. Follow these rules:

        1. Output only the song name or genre and the language (if specified).
        2. Do not include any additional interpretation, explanation, or context.
        3. If the input is unclear or ambiguous, return only the most relevant key terms directly related to the song or genre or print the user input directly if it is not clear.

        Here are examples:

        {few_shot_text}

        Input: {user_input}
        Query:
    """
    messages = [
        SystemMessage(content="You are a music query refinement assistant, specializing in accurate song identification."),
        HumanMessage(content=prompt)
    ]
    response = llm_music.invoke(messages)
    refined_query = response.content.strip()
    print(f"[DEBUG] Refined Query: {refined_query}")
    return refined_query


In [26]:
llm.invoke(refine_music_query("Play the latest song by Taylor Swift."))

[DEBUG] Refined Query: latest song by Taylor Swift


AIMessage(content='As of \\ May 2023, the latest song by Taylor Swift is "Lavender Haze" from her re-recorded album "Speak Now (Taylor\'s Version)". This album is the third in her series of re-recorded albums, following "Fearless (Taylor\'s Version)" and "Red (Taylor\'s Version)". "Lavender Haze" is a synth-pop track that explores the intoxicating feeling of being in love.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 113, 'prompt_tokens': 13, 'total_tokens': 126, 'completion_time': 0.173860197, 'prompt_time': 0.002366817, 'queue_time': 0.02235889, 'total_time': 0.176227014}, 'model_name': 'mixtral-8x7b-32768', 'system_fingerprint': 'fp_c5f20b5bb1', 'finish_reason': 'stop', 'logprobs': None}, id='run-745819b5-24e2-4009-a36b-51613046fb75-0', usage_metadata={'input_tokens': 13, 'output_tokens': 113, 'total_tokens': 126})

In [38]:
import yt_dlp
import vlc
import os

# Specify the VLC path
vlc_path = "C:\ProgramData\Microsoft\Windows\Start Menu\Programs\VideoLAN"  # Replace with the actual path to your VLC installation

# Global music player
music_player = None

def fetch_and_play_music(query):
    """
    Searches for music on YouTube and streams it directly using VLC.
    """
    try:
        # Configure yt-dlp to extract the streaming URL
        ydl_opts = {
            'format': 'bestaudio/best',
            'noplaylist': True,
            'quiet': True,
        }
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            results = ydl.extract_info(f"ytsearch:{query}", download=False)
            if not results.get('entries'):
                return "No results found for your query. Please try a different song."

            # Extract the first result
            result = results['entries'][0]
            video_title = result['title']
            video_url = result['url']

            # Stop any currently playing music
            stop_music()

            # Add headers for VLC to handle YouTube URLs
            media = vlc.Media(video_url)
            media.add_options(
                ":http-user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
                "(KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
                ":http-referrer=https://www.youtube.com/"
            )
            # Play the stream
            global music_player
            # Pass the vlc_path to vlc.Instance
            instance = vlc.Instance(f'--vlc-plugin-path={vlc_path}')
            music_player = instance.media_player_new()
            music_player.set_media(media)
            music_player.audio_set_volume(100)  # Set volume to 100%
            music_player.play()
            music_player.set_media(media)
            music_player.audio_set_volume(100)  # Set volume to 100%
            music_player.play()

            return f"Playing: {video_title}. Say 'Exit' or 'Quit to stop the music."
    except Exception as e:
        return f"An error occurred while playing the song: {e}"


def stop_music():
    """
    Stops the currently playing music, if any.
    """
    global music_player
    if music_player:
        music_player.stop()
        music_player = None


def handle_play_music(user_input):
    """
    Handles user requests for music playback.
    """
    # Handle stop-related commands
    if user_input.lower().strip() in ["stop", "exit", "quit"]:
        stop_music()  # Stop the music playback

    # Refine the query for playing music (if applicable)
    refined_query = user_input.strip()  # For notebook, use raw input directly
    if not refined_query:
        return "I couldn't understand the song you want to play. Could you try rephrasing?"

    # Stop any currently playing music before starting a new one
    stop_music()

    # Fetch and play the song
    return fetch_and_play_music(refined_query)


<>:6: SyntaxWarning: invalid escape sequence '\P'
<>:6: SyntaxWarning: invalid escape sequence '\P'
C:\Users\panka\AppData\Local\Temp\ipykernel_11032\2738795013.py:6: SyntaxWarning: invalid escape sequence '\P'
  vlc_path = "C:\ProgramData\Microsoft\Windows\Start Menu\Programs\VideoLAN"  # Replace with the actual path to your VLC installation


##MainCode Voice Assistant


In [ ]:
import speech_recognition as sr
import pyttsx3
from datetime import datetime
import threading
import requests
import yt_dlp
import vlc
import random
import re
import json
from langchain_groq import ChatGroq
from langchain.schema import SystemMessage, HumanMessage
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory

########################################
# 2) Your unchanged global states & variables
########################################
current_functionality = None
conversation_context = {}
music_player = None

########################################
# 3) TTS (Text-to-Speech) Setup
########################################
tts_engine = pyttsx3.init()
# Adjust speaking rate (200 is fairly normal speed)
tts_engine.setProperty('rate', 200)

def speak(text: str):
    """
    Convert text to speech and also show it (like print).
    """
    print(f"Assistant: {text}")
    tts_engine.say(text)
    tts_engine.runAndWait()

########################################
# 4) STT (Speech-to-Text) Setup
########################################
recognizer = sr.Recognizer()
microphone = sr.Microphone()

def listen_for_command() -> str:
    """
    Listen via microphone and return recognized text.
    """
    try:
        with microphone as source:
            # Optional: adjust for ambient noise
            recognizer.adjust_for_ambient_noise(source, duration=0.5)
            speak("Listening...")
            audio = recognizer.listen(source)
        text = recognizer.recognize_google(audio)
        print(f"You: {text}")
        return text.strip()
    except sr.UnknownValueError:
        return ""
    except sr.RequestError:
        return ""

########################################
# 5) Your unchanged code from the question
#    (We just copy/paste your logic exactly,
#    so that we do not alter it.)
########################################

# -- LLM for intent detection (unchanged) --
llm_intent = ChatGroq(
    api_key="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",
    model="mixtral-8x7b-32768",
    temperature=0.7
)

few_shot_examples = [
    {"input": "Can you explain what AI is?", "intent": "qna"},
    {"input": "Remind me to call John at 3 PM.", "intent": "reminder"},
    {"input": "What's the weather in Paris?", "intent": "weather"},
    {"input": "Play a song from YouTube", "intent": "music"},
]

def detect_intent(user_input):
    """
    Use LLM to detect the intent of the user's query.
    """
    few_shot_text = "\n".join([f"Input: {ex['input']} Intent: {ex['intent']}" for ex in few_shot_examples])
    prompt = f"""
    Classify the user's query into one of these intents:
    - qna
    - reminder
    - weather
    - music

    Examples:
    {few_shot_text}

    Input: {user_input}
    Intent:
    """
    messages = [
        SystemMessage(content="You are an intent detection assistant."),
        HumanMessage(content=prompt)
    ]
    response = llm_intent.invoke(messages)
    intent_detected = response.content.strip().lower()
    print(f"[DEBUG] Detected Intent: {intent_detected}")
    return intent_detected

# -------------
# Next, your process_user_input(...) from the question,
# exactly as written, with your same logic
# (i.e. skipping user_input if inside a functionality, etc.)
# -------------
def process_user_input(user_input):
    """
    Route the user input to the appropriate functionality 
    based on detected intent (unchanged from your question).
    """
    global current_functionality, conversation_context, music_player

    # If inside a functionality, bypass intent detection
    if current_functionality:
        print(f"[DEBUG] Continuing in {current_functionality} functionality.")
        if user_input.lower() == "exit":
            if current_functionality == "music" and music_player:
                music_player.stop()
                music_player = None
            current_functionality = None
            conversation_context.clear()
            return "Exited the current functionality. How can I assist you next?"

        # If you have Q&A, reminder, weather, etc. placeholders:
        if current_functionality == "qna":
            return handle_qna(user_input)
        elif current_functionality == "reminder":
            return handle_reminder(user_input)
        elif current_functionality == "weather":
            return handle_weather(user_input)
        elif current_functionality == "music":
            return handle_play_music(user_input)


    # If not in any functionality, detect the intent
    intent = detect_intent(user_input)
    print(f"[DEBUG] Detected Intent: {intent}")

    # Switch functionalities or default to qna
    if intent in ["qna", "reminder", "weather", "music", "quiz"]:
        current_functionality = intent
        return process_user_input(user_input)
    elif intent == "qna" or intent not in ["reminder", "weather", "music"]:
        current_functionality = "qna"
        return handle_qna(user_input)
    else:
        return "I'm sorry, I couldn't understand that. Could you try rephrasing your request?"

########################################
# 7) The Voice Assistant Main Loop
########################################
def voice_assistant():
    """
    Main voice-based loop that does:
    - TTS for greeting
    - STT to get user input
    - Pass user input to your existing process_user_input
    - TTS the response
    - Repeat until user says "quit"
    """
    speak("Hello! Welcome to your Voice Assistant.")
    speak("You can ask me to set reminders, fetch weather, play music, or ask general questions.")
    speak("Say 'quit' at any time to end our session.")

    while True:
        # Listen for user speech
        user_input = listen_for_command()
        if not user_input:
            speak("Sorry, I didn't catch that. Please try again.")
            continue

        # If user says 'quit'
        if user_input.lower() == "quit":
            speak("Goodbye! Have a great day!")
            break

        # Otherwise, process the input
        response = process_user_input(user_input)
        # Speak the result
        speak(response)


########################################
# 8) Entry point
########################################
if __name__ == "__main__":
    # Just run voice assistant (no text-based chat)
    voice_assistant()

Assistant: Hello! Welcome to your Voice Assistant.
Assistant: You can ask me to set reminders, fetch weather, play music, or ask general questions.
Assistant: Say 'quit' at any time to end our session.
Assistant: Listening...
Assistant: Sorry, I didn't catch that. Please try again.
Assistant: Listening...
Assistant: Sorry, I didn't catch that. Please try again.
Assistant: Listening...
